In [30]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
df = pd.read_csv('location_articles.csv')

In [3]:
X = list(df.body)
Y = list(df.tags)

In [4]:
from ast import literal_eval
Y_ready = []

for sen_tags in Y:
    Y_ready.append(literal_eval(sen_tags))

In [5]:
Y = Y_ready

In [6]:
tags = ['O', 'LOC']

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# cutoff reviews after 110 words
maxlen = 110

# consider the top 36000 words in the dataset
max_words = 36000

# tokenize each sentence in the dataset
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

In [9]:
word_index = tokenizer.word_index
print("Found {} unique tokens.".format(len(word_index)))
ind2word = dict([(value, key) for (key, value) in word_index.items()])

word2id = word_index

# dict. that map each identifier to its word
id2word = {}
for key, value in word2id.items():
    id2word[value] = key

# pad the sequences so that all sequences are of the same size
X_preprocessed = pad_sequences(sequences, maxlen=maxlen, padding='post')

# first example after tokenization and padding. 
X_preprocessed[200]

Found 10489 unique tokens.


array([   1, 1826,  623,    4,  346,  339,    1,  922,  253,   29, 8095,
         18,    6,  689,  436,  999,  143,   49,  926,    7,    6,  421,
          1,   47,   16,    3,    6, 2740,   84,   21, 2575,    8, 1574,
         41,    1,  689,    9,    8,  658, 3304,   18,    6, 5654, 5655,
          5,    6, 8096, 8097,    6,  970,  260,    8,   32, 2115,    7,
          1,  922,   27, 1423,   47,  575,    5, 1417,   83,  903,   22,
         10,    1,  941,    2, 1250,    1, 8098,  230,    7,    1,  922,
          1, 1034,    8,  159,  731,    2,   40,  719,  950,  312,   20,
          6,  960,    4,    1,  922, 1769,  692, 1780,   93,   18,  193,
       2251,    4,  643, 3377,    2,    1, 1911,  256,    7,  226, 2973])

In [10]:
# dict. that map each tag to its identifier
tags2id = {}
for i, tag in enumerate(tags):
    tags2id[tag] = i

# dict. that map each identifier to its tag
id2tag = {}
for key, value in tags2id.items():
    id2tag[value] = key

In [11]:
def preprocess_tags(tags2id, Y_ready):
    
    Y_preprocessed = []
    maxlen = 110
    # for each target 
    for y in Y_ready:
        
        # place holder to store the new preprocessed tag list
        Y_place_holder = []
        
        # for each tag in rhe tag list 
        for tag in y:
            # append the id of the tag in the place holder list
            Y_place_holder.append(tags2id[tag])
        
        # find the lenght of the new preprocessed tag list 
        len_new_tag_list = len(Y_place_holder)
        # find the differance in length between the len of tag list and padded sentences
        num_O_to_add = maxlen - len_new_tag_list
        
        # add 'O's to padd the tag lists
        padded_tags = Y_place_holder + ([tags2id['O']] * num_O_to_add)
        Y_preprocessed.append(padded_tags)
        
    return Y_preprocessed

Y_preprocessed = preprocess_tags(tags2id, Y_ready)

In [15]:
X_preprocessed = np.asarray(X_preprocessed)
Y_preprocessed = np.asarray(Y_preprocessed)

C:\Users\Matthew Kenely\AppData\Roaming\Python\Python37\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [17]:
x_train, x_test, y_train, y_test = train_test_split(X_preprocessed, Y_preprocessed, train_size=0.8, random_state=42)

In [31]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

NameError: name 'X_train' is not defined